# Лабараторная работа №5
## Градиентный бустинг с CatBoost
**Выполнил**: 6131-010402D, Музыка Артем

### Импорт данных

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from catboost import *
from sklearn.metrics import (accuracy_score, PrecisionRecallDisplay, mean_absolute_error, mean_squared_error,
                            plot_roc_curve, roc_auc_score, recall_score, precision_score)
from sklearn.utils import shuffle
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
df = pd.read_csv("data\spotify.csv")

### Обработка данных

Удаление ненужных колонок

In [3]:
for_delete = ['type','uri','track_href','analysis_url','song_name','Unnamed: 0', 'title', 'id']
for f in for_delete:
    df.pop(f)

Создание набора данных и меток классов

In [4]:
genres = list(df["genre"])

def repitless(lst):
    new_list = []
    for g in lst:
        if g not in new_list:
            new_list.append(g)
    return new_list

def str_to_int(lst):
    less = repitless(lst)
    for g in range(len(less)):
        for genre in range(len(lst)):
            if lst[genre] == less[g]:
                lst[genre] = g
    return lst

Y_spotify = str_to_int(list(df["genre"]))
df.pop('genre')

X_spotify = []
for i in range(len(df)):
    X_spotify.append([])
for column in list(df):
    new_list = list(df[column])
    for i in range(len(new_list)):
        X_spotify[i].append(new_list[i])

data_spotify, labels_spotify = shuffle(X_spotify, Y_spotify)

In [5]:
test_size = int(len(data_spotify) * 0.9)
Xtrain = data_spotify[:test_size]
Ytrain = labels_spotify[:test_size]
Xtest = data_spotify[test_size:]
Ytest = labels_spotify[test_size:]

### 1. Применить классификатор ```CatBoostClassifier```
Обратите внимание, что **catboost** умеет работать с категориальными признаками (параметр ```cat_features``` - список индексов категориальных признаков), поэтому использование ```pd.factorize```/```pd.get_dummies``` здесь лишнее.
- вывести итерацию с наилучшим значением **AUC (для бинарной классификации) или наилучшим значением Logloss.
- вывести значения метрик качества классификации на тестовых данных (*accuracy* и *precision* / *recall* / *AUC*, если применимо).
- в случае бинарной классификации на основе CatBoostClassifier построить кривые **PRC** и **ROC**. Вычислить значения **PRC-AUC** и **ROC-AUC**.

In [8]:
cat = CatBoostClassifier(eval_metric="AUC")
cat.fit(Xtrain, Ytrain, eval_set=(Xtest, Ytest), verbose=False)

KeyboardInterrupt: 

In [27]:
print(f"Model is fitted: {model.is_fitted()}")
print(f"Model params: {model.get_params()}")
print(f"Best iteration: {model.get_best_iteration()}")
print(f"Best score: {model.best_score_}")

Model is fitted: True
Model params: {'iterations': 100, 'learning_rate': 0.15, 'random_seed': 63, 'custom_loss': ['AUC', 'Accuracy']}
Best iteration: 99
Best score: {'learn': {'Accuracy': 0.7051531228659977, 'MultiClass': 0.826761694309309}, 'validation': {'Accuracy': 0.6832900023635075, 'AUC:type=Mu': 0.9739030307057599, 'MultiClass': 0.8942949236761298}}


In [28]:
pred = model.predict(Xtest_spotify)

In [29]:
print(f"Accuracy: {accuracy_score(Ytest_spotify, pred)}")
print(f"Precision: {precision_score(Ytest_spotify, pred, average='weighted')}")
print(f"Recall: {recall_score(Ytest_spotify, pred, average='weighted')}")

Accuracy: 0.6828173008744978
Precision: 0.6890007111815011
Recall: 0.6828173008744978


In [30]:
plot_roc_curve(model, Xtest_spotify, Ytest_spotify)

ValueError: Expected 'estimator' to be a binary classifier, but got CatBoostClassifier fit on multiclass (15 classes) data

### 2. Применить регрессию `CatBoostRegressor` к набору данных для регрессии.
Вывести номер итерации и значения метрик (**MAE** и **MSE**) для наилучшей итерации. Обратите внимание, что в задаче восстановления регрессии целевая переменная имеет непрерывные значения.